<a href="https://colab.research.google.com/github/KGuzikowski/AlgTrading/blob/master/Simple_indicators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h516909a_0.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py37ha21ca33_2.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.7/dist-packages/ lib/python3.7/site-packages/talib --strip-components=3

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3781    0  3781    0     0  14108      0 --:--:-- --:--:-- --:--:-- 14055
100  503k  100  503k    0     0   807k      0 --:--:-- --:--:-- --:--:--  807k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3769    0  3769    0     0  18475      0 --:--:-- --:--:-- --:--:-- 18475
100  406k  100  406k    0     0   750k      0 --:--:-- --:--:-- --:--:-- 1891k


In [4]:
# !pip install TA-Lib
!pip install python-binance
!pip install binance-client

In [235]:
%matplotlib inline

import binance.client
import pandas as pd
import numpy as np
import time
import os
import requests
from pprint import pprint
import subprocess
import datetime as dt
import json
import pandas as pd
import talib

import matplotlib.pyplot as plt
import seaborn as sns

# from xgboost import XGBClassifier
from tqdm import tqdm

# Simple Binance Data Fetch

In [236]:
class BinanceClient():
    column_names = [
        'open_time',
        'open',
        'high',
        'low',
        'close',
        'vol',
        'close_time',
        'quote_asset_volume',
        'number_of_trades',
        'taker_buy_base_asset_volume',
        'taker_buy_quote_asset_volume',
        'ignore'
    ]

    def __init__(self, api_key, api_secret, interval):
        self.client = binance.client.Client(api_key, api_secret)
        self.INTERVAL = interval

    def polish_kline(self, raw_kline):
        kline_d =  {
            'open_time': raw_kline[0],
            'open': raw_kline[1],
            'close': raw_kline[2],
            'low': raw_kline[3],
            'high': raw_kline[4],
            'vol': raw_kline[5]
        }
        kline_d['open_datetime'] = pd.to_datetime(kline_d['open_time'], unit='ms')
        return kline_d

    def historical_klines_generator(self, instrument, interval, start_dt, end_dt=None):
        for kline in self.client.get_historical_klines_generator(instrument, interval, str(start_dt), str(end_dt) if end_dt else None):
            yield self.polish_kline(kline)


In [237]:
api_key = "vT0XoiF4FbrpHbH04ZEew6A9jOORd79J4RdmId0CTT6wwX97EuY9hPCBcp9F904f"
api_secret = "wcpF4Ne1tNGbq5vPx2oaXrpbE2JnNXhn6tKV6I32uKNdBMuEazRfDPiMJkIVqVr8"

In [238]:
client = BinanceClient(api_key, api_secret, binance.client.Client.KLINE_INTERVAL_5MINUTE)

start_date = dt.datetime(2021,10,7,10,0,0)
instrument = "BTCUSDT"

kline_df = pd.DataFrame(columns=["open","high","low","close","vol"])
for kline in client.historical_klines_generator(instrument, client.INTERVAL, start_date):
    kline_df.loc[kline['open_datetime']] = (kline["open"], kline["high"], kline["low"], kline["close"], kline["vol"])


In [239]:
kline_df

,open,high,low,close,vol
2021-10-07 10:00:00,54300.57000000,54339.51000000,54240.98000000,54432.49000000,191.72031000
2021-10-07 10:05:00,54339.51000000,54263.05000000,54200.00000000,54339.51000000,175.63492000
2021-10-07 10:10:00,54263.04000000,54142.79000000,54100.01000000,54263.05000000,213.18491000
2021-10-07 10:15:00,54143.84000000,54135.28000000,54088.39000000,54199.99000000,177.47716000
2021-10-07 10:20:00,54135.28000000,54102.98000000,54046.67000000,54166.85000000,183.81866000
...,...,...,...,...,...
2021-10-20 20:15:00,66011.96000000,65911.40000000,65716.04000000,66199.92000000,501.41978000
2021-10-20 20:20:00,65904.21000000,65833.33000000,65690.00000000,66002.35000000,246.23305000
2021-10-20 20:25:00,65833.33000000,65705.81000000,65705.80000000,65974.34000000,182.37532000
2021-10-20 20:30:00,65705.81000000,65799.98000000,65662.95000000,65799.99000000,243.64460000


# Bollinger Bands

In [189]:
# dla co 5 min 2880 to 20 dni w ty

BB = talib.BBANDS(
    kline_df.close,
    timeperiod=700,
    nbdevup=3,
    nbdevdn=2,
    matype=0,
)

In [190]:
bb_bands = ['upper', 'middle', 'lower']

In [191]:
boll_df = pd.DataFrame(index=kline_df.index)
boll_df["price"] = kline_df.close.astype(float)
boll_df = boll_df.assign(**dict(zip(bb_bands, BB)))
boll_df["ma"] = boll_df.price.rolling(300).mean()
boll_df["std"] = boll_df.price.rolling(300).std()

In [192]:
boll_df

,price,upper,middle,lower,ma,std
2021-10-07 10:00:00,54432.49,NaN,NaN,NaN,NaN,NaN
2021-10-07 10:05:00,54339.51,NaN,NaN,NaN,NaN,NaN
2021-10-07 10:10:00,54263.05,NaN,NaN,NaN,NaN,NaN
2021-10-07 10:15:00,54199.99,NaN,NaN,NaN,NaN,NaN
2021-10-07 10:20:00,54166.85,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2021-10-20 16:50:00,66545.55,66963.514455,62985.651729,60333.743244,64266.212600,962.226272
2021-10-20 16:55:00,66477.15,66988.303981,62991.675729,60327.256894,64278.938267,966.142410
2021-10-20 17:00:00,66487.99,67013.165896,62997.497100,60320.384569,64292.265667,968.951811
2021-10-20 17:05:00,66321.80,67035.830736,63003.054871,60314.537628,64305.068233,970.312985


In [215]:
boll_df["lower_cross_from_top_signal"] = (boll_df.price.shift(1) > boll_df.lower.shift(1)) & (boll_df.price < boll_df.lower)
boll_df["upper_cross_from_bottom_signal"] = (boll_df.price.shift(1) < boll_df.upper.shift(1)) & (boll_df.price > boll_df.upper)

In [197]:
boll_df[boll_df.lower_cross_from_top_signal]

,price,upper,middle,lower,ma,std,lower_cross_from_top_signal,upper_cross_from_bottom_signal,lower_cross_from_bottom_signal,upper_cross_from_top_signal
2021-10-13 07:30:00,54965.71,58885.930349,56562.039686,55012.779243,56430.689400,672.998991,True,False,False,False
2021-10-13 08:40:00,54956.14,58874.041863,56564.440014,55024.705449,56318.324100,702.976820,True,False,False,False
2021-10-13 10:40:00,54948.01,58868.267471,56565.644300,55030.562186,56112.609800,723.222372,True,False,False,False
2021-10-13 12:50:00,54977.46,58914.599120,56554.720014,54981.467277,55928.475000,705.316227,True,False,False,False
2021-10-13 13:55:00,54927.00,58951.721572,56540.870729,54933.636833,55844.423100,677.581290,True,False,False,False
2021-10-17 20:20:00,59700.88,63037.584067,61071.668671,59761.058408,60945.032967,226.448141,True,False,False,False
2021-10-17 20:30:00,59718.57,63029.550844,61072.990229,59768.616485,60936.393800,246.041088,True,False,False,False


In [217]:
import plotly.graph_objects as go


fig = go.Figure()
fig.add_trace(go.Scatter(x=boll_df.index, y=boll_df.price, mode="lines", name="price"))
fig.add_trace(go.Scatter(x=boll_df.index, y=boll_df.ma, mode="lines", name="ma"))
fig.add_trace(go.Scatter(x=boll_df.index, y=boll_df.upper, mode="lines", line=dict(color="firebrick"), name="upper Bollinger"))
fig.add_trace(go.Scatter(x=boll_df.index, y=boll_df.lower, mode="lines", line=dict(color="firebrick"), name="lower Bollinger"))
fig.add_trace(go.Scatter(x=boll_df[boll_df.lower_cross_from_top_signal].index, y=boll_df[boll_df.lower_cross_from_top_signal].price, mode="markers", marker=dict(size=12, color="hotpink"), name="Bollinger lower cross from top"))
fig.add_trace(go.Scatter(x=boll_df[boll_df.upper_cross_from_bottom_signal].index, y=boll_df[boll_df.upper_cross_from_bottom_signal].price, mode="markers", marker=dict(size=12, color="green"), name="Bollinger upper cross from bottom"))


fig.show()

## Wnioski

Na "Bollinger lower cross from top" zacznij kupować. **Czyli jeśli poniżej lower to kupuj.**

Na "Bollinger upper cross from bottom" sprzedaj. **Czyli jeśli ponad upper to sprzedawaj.**

# MACD

In [206]:
ma_df = pd.DataFrame(index=kline_df.index)
ma_df["price"] = kline_df.close.astype(float)

## MA

In [212]:
ma_df["maS"] = talib.MA(ma_df["price"], timeperiod=200, matype=0)
ma_df["stdS"] = ma_df.price.rolling(200).std()
ma_df["maB"] = talib.MA(ma_df["price"], timeperiod=600, matype=0)

In [219]:
ma_df["ma_small_cross_from_bottom"] = (ma_df.maS.shift(1) < ma_df.maB.shift(1)) & ((ma_df.maS > ma_df.maB))
ma_df["ma_small_cross_from_top"] = (ma_df.maS.shift(1) > ma_df.maB.shift(1)) & ((ma_df.maS < ma_df.maB))

In [224]:
import plotly.graph_objects as go


fig = go.Figure()
fig.add_trace(go.Scatter(x=ma_df.index, y=ma_df.price, mode="lines", name="price"))
fig.add_trace(go.Scatter(x=ma_df.index, y=ma_df.maS, mode="lines", name="ma 300"))
fig.add_trace(go.Scatter(x=ma_df.index, y=ma_df.maB, mode="lines", name="ma 700"))
fig.add_trace(go.Scatter(x=ma_df[ma_df.ma_small_cross_from_bottom].index, y=ma_df[ma_df.ma_small_cross_from_bottom].maS, mode="markers", marker=dict(size=12, color="green"), name="ma small cross signal from bottom"))
fig.add_trace(go.Scatter(x=ma_df[ma_df.ma_small_cross_from_bottom].index, y=ma_df[ma_df.ma_small_cross_from_bottom].price, mode="markers", marker=dict(size=12, color="firebrick"), name="ma small cross signal from bottom - price"))
fig.add_trace(go.Scatter(x=ma_df[ma_df.ma_small_cross_from_top].index, y=ma_df[ma_df.ma_small_cross_from_top].maS, mode="markers", marker=dict(size=12, color="hotpink"), name="ma small cross signal from top"))
fig.add_trace(go.Scatter(x=ma_df[ma_df.ma_small_cross_from_top].index, y=ma_df[ma_df.ma_small_cross_from_top].price, mode="markers", marker=dict(size=12, color="yellowgreen"), name="ma small cross signal from top - price"))


fig.show()

## EMA

In [232]:
ma_df["maS"] = talib.EMA(ma_df["price"], timeperiod=200)
ma_df["stdS"] = ma_df.price.rolling(200).std()
ma_df["maB"] = talib.EMA(ma_df["price"], timeperiod=600)

In [233]:
ma_df["ma_small_cross_from_bottom"] = (ma_df.maS.shift(1) < ma_df.maB.shift(1)) & ((ma_df.maS > ma_df.maB))
ma_df["ma_small_cross_from_top"] = (ma_df.maS.shift(1) > ma_df.maB.shift(1)) & ((ma_df.maS < ma_df.maB))

In [234]:
import plotly.graph_objects as go


fig = go.Figure()
fig.add_trace(go.Scatter(x=ma_df.index, y=ma_df.price, mode="lines", name="price"))
fig.add_trace(go.Scatter(x=ma_df.index, y=ma_df.maS, mode="lines", name="ma 300"))
fig.add_trace(go.Scatter(x=ma_df.index, y=ma_df.maB, mode="lines", name="ma 700"))
fig.add_trace(go.Scatter(x=ma_df[ma_df.ma_small_cross_from_bottom].index, y=ma_df[ma_df.ma_small_cross_from_bottom].maS, mode="markers", marker=dict(size=12, color="green"), name="ma small cross signal from bottom"))
fig.add_trace(go.Scatter(x=ma_df[ma_df.ma_small_cross_from_bottom].index, y=ma_df[ma_df.ma_small_cross_from_bottom].price, mode="markers", marker=dict(size=12, color="firebrick"), name="ma small cross signal from bottom - price"))
fig.add_trace(go.Scatter(x=ma_df[ma_df.ma_small_cross_from_top].index, y=ma_df[ma_df.ma_small_cross_from_top].maS, mode="markers", marker=dict(size=12, color="hotpink"), name="ma small cross signal from top"))
fig.add_trace(go.Scatter(x=ma_df[ma_df.ma_small_cross_from_top].index, y=ma_df[ma_df.ma_small_cross_from_top].price, mode="markers", marker=dict(size=12, color="yellowgreen"), name="ma small cross signal from top - price"))


fig.show()

## Wnioski

### EMA lepsze niż MA!

`ma small cross signal from bottom"` - kupuj

`ma small cross signal from top` - sprzedawaj




# ADX - Average directional index

In [317]:
df = pd.DataFrame(index=kline_df.index)
df["price"] = kline_df.close.astype(float)
df['ADX0'] = talib.ADX(
    kline_df.high,
    kline_df.low,
    kline_df.close,
    timeperiod=20
)
df["ADX0var"] = talib.VAR(df.ADX0, timeperiod=20, nbdev=1)
df['ADX1'] = talib.ADX(
    kline_df.high,
    kline_df.low,
    kline_df.close,
    timeperiod=100
)
df["ADX1var"] = talib.VAR(df.ADX1, timeperiod=20, nbdev=1)
df['ADX2'] = talib.ADX(
    kline_df.high,
    kline_df.low,
    kline_df.close,
    timeperiod=700
)
df["ADX2var"] = talib.VAR(df.ADX2, timeperiod=20, nbdev=1)
df

,price,ADX0,ADX0var,ADX1,ADX1var,ADX2,ADX2var
2021-10-07 10:00:00,54432.49,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-07 10:05:00,54339.51,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-07 10:10:00,54263.05,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-07 10:15:00,54199.99,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-07 10:20:00,54166.85,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2021-10-20 20:15:00,66199.92,20.394185,15.193927,22.504388,0.058467,9.781273,0.000912
2021-10-20 20:20:00,66002.35,21.364744,14.084776,22.414164,0.043300,9.784200,0.000864
2021-10-20 20:25:00,65974.34,22.286775,12.728625,22.324843,0.031987,9.787123,0.000811
2021-10-20 20:30:00,65799.99,22.705873,11.387662,22.247810,0.025178,9.790268,0.000757


## ADX 20

In [321]:
import plotly.graph_objects as go


fig = go.Figure()
fig.add_trace(go.Scatter(x=ma_df.index, y=df.price, mode="lines", name="price"))
fig.show()

In [322]:
import plotly.graph_objects as go


fig = go.Figure()
fig.add_trace(go.Scatter(x=ma_df.index, y=df.ADX0var, mode="lines", name="ADX variance"))
fig.show()

In [323]:
import plotly.graph_objects as go


fig = go.Figure()
fig.add_trace(go.Scatter(x=ma_df.index, y=df.ADX0, mode="lines", name="ADX"))
fig.show()

## ADX 100

In [324]:
import plotly.graph_objects as go


fig = go.Figure()
fig.add_trace(go.Scatter(x=ma_df.index, y=df.price, mode="lines", name="price"))
fig.show()

In [325]:
import plotly.graph_objects as go


fig = go.Figure()
fig.add_trace(go.Scatter(x=ma_df.index, y=df.ADX1var, mode="lines", name="ADX variance"))
fig.show()

In [326]:
import plotly.graph_objects as go


fig = go.Figure()
fig.add_trace(go.Scatter(x=ma_df.index, y=df.ADX1, mode="lines", name="ADX"))
fig.show()

## ADX 700

In [327]:
import plotly.graph_objects as go


fig = go.Figure()
fig.add_trace(go.Scatter(x=ma_df.index, y=df.price, mode="lines", name="price"))
fig.show()

In [328]:
import plotly.graph_objects as go


fig = go.Figure()
fig.add_trace(go.Scatter(x=ma_df.index, y=df.ADX2var, mode="lines", name="ADX2 variance"))
fig.show()

In [329]:
import plotly.graph_objects as go


fig = go.Figure()
fig.add_trace(go.Scatter(x=ma_df.index, y=df.ADX2, mode="lines", name="ADX2"))
fig.show()

## Wnioski

Muszę użyć ADX dwa razy:
- mniejszy okres - bliższe trendy
- większy okres - dłuższe trendy

Zamiast ADX użyć jego wariancji, ponieważ duże ADX